In [1]:
import gradio as gr
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image, ImageEnhance
from itertools import permutations, combinations
import numpy as np
import uuid

In [2]:
from roboflow import Roboflow
rf = Roboflow(api_key="Q0EN6hdjVCdwqJnAAYvc")
project = rf.workspace().project("playing-cards-ow27d")
model = project.version(2).model

loading Roboflow workspace...
loading Roboflow project...


In [3]:
path_52cards = "52cards/"
prefix = "800px-Playing_card_"
suffix = ".svg.png"

In [4]:
suits = ["spade", "club", "diamond", "heart"]
cards = ["2", "3", "4", "5", "6", "7", "8", "9", "10", "J", "Q", "K", "A"]
ranks = {"0_highcard": 0, "0_pair": 1, "1_highcard": 2, "1_pair": 3,
         "2_highcard": 4, "2_pair": 5, "3_highcard": 6, "3_pair": 7,
         "4_highcard": 8, "4_pair": 9, "5_highcard": 10, "5_pair": 11,
         "6_highcard": 12, "6_pair": 13, "7_highcard": 14, "7_pair": 15,
         "8_highcard": 16, "8_pair": 17, "9_highcard": 18, "9_pair": 19,
         "three_of_images": 20, "straight": 21, "three_of_a_kind": 22
        }
all_of_straights = list(permutations(["A", "2", "3"]))
all_of_straights += list(permutations(["2", "3", "4"]))
all_of_straights += list(permutations(["3", "4", "5"]))
all_of_straights += list(permutations(["4", "5", "6"]))
all_of_straights += list(permutations(["5", "6", "7"]))
all_of_straights += list(permutations(["6", "7", "8"]))
all_of_straights += list(permutations(["7", "8", "9"]))
all_of_straights += list(permutations(["8", "9", "10"]))
all_of_straights += list(permutations(["9", "10", "J"]))
all_of_straights += list(permutations(["10", "J", "Q"]))
all_of_straights += list(permutations(["J", "Q", "K"]))
all_of_straights += list(permutations(["Q", "K", "A"]))

def intt(card):
    try:
        num = int(card)
    except:
        if card == "A":
            return 1
        else:
            return 0
    return num % 10


def card_split(suit_card):
    tmp = suit_card.split("_")
    return tmp[0], tmp[1]


def cards_comparison(card_1, card_2):
    s1, c1 = card_split(card_1)
    s2, c2 = card_split(card_2)
    c_in_1, c_in_2 = cards.index(c1), cards.index(c2)
    if c_in_1 > c_in_2:
        return 1
    if c_in_1 < c_in_2:
        return -1
    # c_in_1 == c_in_2:
    s_in_1, s_in_2 = suits.index(s1), suits.index(s2)
    if s_in_1 > s_in_2:
        return 1
    if s_in_1 < s_in_2:
        return -1
    return 0


def four_of_a_kind(hands):
    count_ = dict()
    for card in hands:
        s, c = card_split(card)
        if c not in count_.keys():
            count_[c] = 1
        else:
            count_[c] += 1
    kind = None
    for key, value in count_.items():
        if value == 4:
            if kind == None:
                kind = key
            elif cards.index(key) > cards.index(kind):
                kind = key
    return kind


A23_combo = list(permutations(["A", "2", "3"]))
def identify_combo(combo):
    card_1, card_2, card_3 = combo
    s1, c1 = card_split(card_1)
    s2, c2 = card_split(card_2)
    s3, c3 = card_split(card_3)

    compare_1_2 = cards_comparison(card_1, card_2)
    compare_2_3 = cards_comparison(card_2, card_3)
    compare_3_1 = cards_comparison(card_3, card_1)
    
    if c1 == c2 == c3:
        return "three_of_a_kind", card_1

    if (c1, c2, c3) in all_of_straights:
        if (c1, c2, c3) in A23_combo:
            if c1 == "3":
                return "straight", card_1
            if c2 == "3":
                return "straight", card_2
            if c3 == "3":
                return "straight", card_3

        if compare_1_2 == 1 and compare_3_1 == -1:
            return "straight", card_1
        if compare_2_3 == 1 and compare_1_2 == -1:
            return "straight", card_2
        if compare_3_1 == 1 and compare_2_3 == -1:
            return "straight", card_3
    
    # There's no such thing as "J", "Q" and "K" appearing together
    if c1 in ["J", "Q", "K"] and c2 in ["J", "Q", "K"] and c3 in ["J", "Q", "K"]:
        if c1 == c2:
            if compare_1_2 == 1:
                return "three_of_images", card_1
            else:
                return "three_of_images", card_2
        if c2 == c3:
            if compare_2_3 == 1:
                return "three_of_images", card_2
            else:
                return "three_of_images", card_3
        if c3 == c1:
            if compare_3_1 == 1:
                return "three_of_images", card_3
            else:
                return "three_of_images", card_1
    
    cards_sum = (intt(c1) + intt(c2) + intt(c3)) % 10
    if c1 == c2:
        if compare_1_2 == 1:
            return str(cards_sum) + "_pair", card_1
        else:
            return str(cards_sum) + "_pair", card_2
    if c2 == c3:
        if compare_2_3 == 1:
            return str(cards_sum) + "_pair", card_2
        else:
            return str(cards_sum) + "_pair", card_3
    if c3 == c1:
        if compare_3_1 == 1:
            return str(cards_sum) + "_pair", card_3
        else:
            return str(cards_sum) + "_pair", card_1
    if compare_1_2 == 1 and compare_3_1 == -1:
        return str(cards_sum) + "_highcard", card_1
    if compare_2_3 == 1 and compare_1_2 == -1:
        return str(cards_sum) + "_highcard", card_2
    if compare_3_1 == 1 and compare_2_3 == -1:
        return str(cards_sum) + "_highcard", card_3


def comboes_comparison(combo_1, combo_2):
    identify_1, repr_1 = identify_combo(combo_1)
    identify_2, repr_2 = identify_combo(combo_2)
    i_in_1 = ranks[identify_1]
    i_in_2 = ranks[identify_2]
    if i_in_1 > i_in_2:
        return 1
    if i_in_1 < i_in_2:
        return -1
    # i_in_1 == i_in 2:
    return cards_comparison(repr_1, repr_2)


def hands_comparison(hands_1, hands_2):
    combo_1_1, combo_1_2, combo_1_3 = hands_1
    combo_2_1, combo_2_2, combo_2_3 = hands_2
    value = comboes_comparison(combo_1_1, combo_2_1)
    value += comboes_comparison(combo_1_2, combo_2_2)
    value += comboes_comparison(combo_1_3, combo_2_3)
    return value


def vip_permutations(hands):
    vip_perm = []
    comboes_1 = list(combinations(hands, 3))
    for c0, c1, c2 in comboes_1:
        combo_1 = (c0, c1, c2)
        tmp_hands = hands.copy()
        tmp_hands.remove(c0)
        tmp_hands.remove(c1)
        tmp_hands.remove(c2)
        comboes_2 = list(combinations(tmp_hands, 3))
        for c3, c4, c5 in comboes_2:
            combo_2 = (c3, c4, c5)

            combo_3 = tmp_hands.copy()
            combo_3.remove(c3)
            combo_3.remove(c4)
            combo_3.remove(c5)
            combo_3 = tuple(combo_3)

            if comboes_comparison(combo_1, combo_2) == -1 and comboes_comparison(combo_2, combo_3) == -1:
                vip_hands = [combo_1, combo_2, combo_3]
                vip_perm.append(vip_hands)
    return vip_perm


def n_largest(a):
    b1 = [0] * len(a)
    b3 = [0] * len(a) 
    for i in range(len(a)):
        for j in range(len(a)):
            if i != j:
                tmp = hands_comparison(a[i], a[j])
                b1[i] += tmp // np.abs(tmp)
                b3[i] += tmp

    b1 = np.array(b1)
    n_top = 7
    ind = np.argpartition(b1, -n_top)[-n_top:]

    top_10 = b1[ind]
    for i in range(0, n_top-1):
        for j in range(0, n_top-i-1):
            if top_10[j] < top_10[j+1]:
                top_10[j], top_10[j+1] = top_10[j+1], top_10[j]
                ind[j], ind[j+1] = ind[j+1], ind[j]
            if top_10[j] == top_10[j+1]:
                if hands_comparison(a[j], a[j+1]) < 0:
                    top_10[j], top_10[j+1] = top_10[j+1], top_10[j]
                    ind[j], ind[j+1] = ind[j+1], ind[j]

    tmp = [_/(len(a)-1) for _ in top_10]

    return np.array(a)[ind], tmp

In [5]:
test_path = "test/"
results_path = "results/"

def show_hand(hand, result_path):
    fig = plt.figure(figsize=(6, 6))
    columns, rows = 3, 3
    for i in range(3):
        for j in range(3):
            card = hand[i][j]
            card_path = path_52cards + prefix + card + suffix
            img = mpimg.imread(card_path)
            fig.add_subplot(rows, columns, i*3 + j + 1)
            plt.axis("off")
            plt.imshow(img)
    fig.savefig(result_path)
    im = Image.open(result_path)
    plt.axis("off")
    plt.show()
    return im
    

def flip(img):
    uuid_generator = str(uuid.uuid4())
    image_path = test_path + uuid_generator + ".jpg"
    result_path = results_path + uuid_generator + ".jpg"
    
    width, height = img.size
    if width == 0 or height == 0:
        return img
    
    width = width // (height // 800)
    height = 800
    img = img.resize((width, height))
    
    enhancer = ImageEnhance.Brightness(img)
    factor = 1.5
    img = enhancer.enhance(factor)
    
    img_save = img.save(image_path)
    image_test = model.predict(image_path, confidence=40, overlap=70).json()
    
    hands = []
    preds = image_test["predictions"]
    for i, pred in enumerate(preds):
        cl = pred["class"]
        card = cl[:-1]
        if cl[-1] == "S":
            card = "spade_" + card
        elif cl[-1] == "C":
            card = "club_" + card
        elif cl[-1] == "D":
            card = "diamond_" + card
        elif cl[-1] == "H":
            card = "heart_" + card
        hands.append(card)
    hands = list(set(hands))
    if len(hands) < 9:
        return img
    hands = hands[:9]
    
    kind = four_of_a_kind(hands)
    if kind:
        fig = plt.figure(figsize=(12, 3))
        four = [_ + "_" + kind for _ in suits]
        for i, card in enumerate(four):
            card_path = path_52cards + prefix + card + suffix
            img = mpimg.imread(card_path)
            fig.add_subplot(1, 4, i+1)
            plt.axis("off")
            plt.imshow(img)
        fig.savefig(result_path)
        im = Image.open(result_path)
        plt.axis("off")
        plt.show()
        return im
    
    all_hands = vip_permutations(hands)
    n_lar, scores = n_largest(all_hands)
    try:
        for hand, score in zip(n_lar, scores):
            im = show_hand(hand, result_path)
            break
    except Exception as e:
        print(e)
        im = img
    return im

In [6]:
# demo = gr.Interface(
#     flip, 
#     inputs=[gr.inputs.Image(type="pil", label="Input Image", source="webcam"),], 
#     outputs="image",
#     live=True
# )

demo = gr.Interface(
    flip, 
    inputs=[gr.inputs.Image(type="pil", label="Input Image"),], 
    outputs="image",
)

demo.launch(share=True)

Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
`optional` parameter is deprecated, and it has no effect


Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://e72d668a-cb71-4ccc.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces
